In [1]:
import pandas as pd

In [2]:
pd.read_json('../dataset/train.json')

,uid,subj,rel_id,input,truncated_input,output
0,e54a5932-7bfd-4b50-a971-fe1fb5640651,Tokyo Metropolitan Bureau of Transportation,P740,Tokyo Metropolitan Bureau of Transportation wa...,Tokyo Metropolitan Bureau of Transportation wa...,Tokyo
1,9964b31d-4b2c-497e-a1b8-720d7234299e,Belle du Seigneur,P495,Belle du Seigneur was created in [MASK] .,Belle du Seigneur was created in,Switzerland
2,5f813b71-1f38-46af-a87d-e1dc3b5bcae5,chocolate,P527,chocolate consists of [MASK] .,chocolate consists of,sugar
3,2c86fe62-2f04-42b3-a526-c7b8d71efaef,Vientiane,P1376,Vientiane is the capital of [MASK] .,Vientiane is the capital of,Laos
4,190697f3-9e00-4f83-a9e1-d7264d52f6eb,Andorra,P530,Andorra maintains diplomatic relations with [M...,Andorra maintains diplomatic relations with,Italy
...,...,...,...,...,...,...
21244,8b951418-ae04-4c03-924e-1e82bac5098f,Klaus Tschira,P108,Klaus Tschira works for [MASK] .,Klaus Tschira works for,IBM
21245,1530e3bb-f9a9-4076-8de7-b72d74e8ce34,Anna Louisa Karsch,P106,Anna Louisa Karsch is a [MASK] by profession .,Anna Louisa Karsch is a,poet
21246,ebbc8828-f162-4037-a827-ba8681b2a434,Texas,P36,The capital of Texas is [MASK] .,The capital of Texas is,Austin
21247,fbddd8cf-954c-4596-b928-5b83305e995d,Reasonable Doubts,P449,Reasonable Doubts was originally aired on [MAS...,Reasonable Doubts was originally aired on,NBC


In [3]:
import json    
with open("../dataset/intervention_data/gpt2_train=True_entropy.json",'rb') as f:
    ie_data = json.load(f)

In [9]:
import torch
torch.arange(1000).view(10,10,10).max(axis=2).values

tensor([[  9,  19,  29,  39,  49,  59,  69,  79,  89,  99],
        [109, 119, 129, 139, 149, 159, 169, 179, 189, 199],
        [209, 219, 229, 239, 249, 259, 269, 279, 289, 299],
        [309, 319, 329, 339, 349, 359, 369, 379, 389, 399],
        [409, 419, 429, 439, 449, 459, 469, 479, 489, 499],
        [509, 519, 529, 539, 549, 559, 569, 579, 589, 599],
        [609, 619, 629, 639, 649, 659, 669, 679, 689, 699],
        [709, 719, 729, 739, 749, 759, 769, 779, 789, 799],
        [809, 819, 829, 839, 849, 859, 869, 879, 889, 899],
        [909, 919, 929, 939, 949, 959, 969, 979, 989, 999]])

In [1]:
import pandas as pd
import numpy as np
from functools import partial
from copy import deepcopy
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

/home/sjlee/miniconda3/envs/sj_virtual/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
def make_content(examples):
    examples['content'] = examples['truncated_input'] + ' ' + examples['output'] 
    return examples

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [19]:
def preprocess_function(examples,tokenizer):
    # batch=True를 통해 하나의 리스트로써 들어온다
    examples = examples['content']
    tokenized = tokenizer(examples, padding=True,truncation=True,return_tensors = 'pt')
    return tokenized


In [20]:
train = load_dataset("json", data_files = "../dataset/train.json")
valid = load_dataset("json", data_files = "../dataset/validation.json")
preprocess_func = partial(preprocess_function,tokenizer=tokenizer)

train = train.map(make_content)
valid = valid.map(make_content)


train_dataset = train.map(preprocess_func,
                    batched = True,
                    num_proc = 4,
                    remove_columns = train['train'].column_names
                    ) 

Map (num_proc=4): 100%|██████████| 21249/21249 [00:07<00:00, 2798.87 examples/s]


In [8]:
tokenizer(' Tokyo Metropolitan Bureau of Transportation')

{'input_ids': [11790, 21609, 9840, 286, 15198], 'attention_mask': [1, 1, 1, 1, 1]}

In [23]:
tokenizer.decode([15198])
tokenizer.encode(' Transportation')

[15198]

In [21]:
tokenizer.encode('Transportation')

[8291, 10189]

In [24]:
tokenizer.decode([8226])

'yo'

In [22]:
train_dataset['train']['input_ids'][0]

[19042,
 8226,
 21609,
 9840,
 286,
 15198,
 373,
 9393,
 287,
 11790,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256]

In [1]:
"Father Lies Me".capitalize()

'Father lies me'

In [2]:
' Zagreb is the capital of'.split()

['Zagreb', 'is', 'the', 'capital', 'of']

In [4]:
[1,2,3,4,3].count(3)

2

In [8]:
tokenizer(' Lies my father told me was created in')

{'input_ids': [35337, 616, 2988, 1297, 502, 373, 2727, 287], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [33]:
tokenizer.encode('canada')

[5171, 4763]

In [35]:
tokenizer.encode('Entity')

[32398]

In [27]:
tokenizer.decode(32398)

'Entity'

In [28]:
tokenizer.decode(20885)

' Entity'

In [26]:
A = np.array([1,2,3,4])
A[0]

1